## Computer Vision Part

In [2]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [31]:
# Import libraries
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model


In [4]:
# Pygame for alert sound
pip install pygame


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [32]:
# Define Cascade Classifier
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
model = load_model('C__Users_Admin_Desktop_drowsiness detection_model_drowsiness_model3.h5')

In [33]:
from pygame import mixer

In [35]:
mixer.init()
sound= mixer.Sound("WhatsApp Audio 2023-02-05 at 5.59.42 PM.mpeg")
score=0
cap=cv2.VideoCapture(0)   #For video capture
if not cap.isOpened():
  print("Error: VideoCapture not opened")
  exit()

while True:                 
  ret, frame = cap.read()
  height,width = frame.shape[0:2]
  if frame is None:
    print("Error: frame is empty")
    break

  if face_cascade.empty():
    print("Error: Cascade classifier is empty")
    break
    
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

  faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)  # For face
  eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)  # For eyes
  
  cv2.rectangle(frame,(0,height-50),(200,height),(0,0,0),thickness=cv2.FILLED)
  
  for (x,y,w,h) in faces:
    cv2.rectangle(frame, pt1=(x,y), pt2=(x+w,y+h), color=(255,0,0), thickness=3)

  for (ex,ey,ew,eh) in eyes:
    cv2.rectangle(frame, pt1=(ex,ey), pt2=(ex+ew,ey+eh), color=(255,0,0), thickness=3)
    
    # image preprocessing after extracting the image from frame
    eye=frame[ey:ey+eh,ex:ex+w]
    eye=cv2.resize(eye,(125,125))
    eye=eye/255
    eye=eye.reshape(125,125,3)
    eye=np.expand_dims(eye,axis=0)
    
    #predict the image 
    prediction=model.predict(eye)
    
    #eyes are closed
    if prediction[0][0]>0.50:
        cv2.putText(frame,'Closed',(10,height-20),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    fontScale=1, 
                    color=(255,255,255), 
                    thickness=1)
        cv2.putText(frame,'score'+str(score),(100,height-20),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    fontScale=0.5, 
                    color=(255,255,255), 
                    thickness=1)
        score=score+1
        if(score>15):
            try:
                sound.play()
            except:
                pass
            
       
      
    #eyes are open
    elif prediction[0][1]>0.5:
        cv2.putText(frame,'Open',(10,height-20),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    fontScale=1, 
                    color=(255,255,255), 
                    thickness=1)
        cv2.putText(frame,'score'+str(score),(100,height-20),
                    fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                    fontScale=0.5, 
                    color=(255,255,255), 
                    thickness=1)
        score=score-1
        if(score<0):
            score=0
    
  cv2.imshow('frame',frame)
  if cv2.waitKey(33) & 0xFF==ord('q'):
    break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 115ms/step
